Loading the json with the grade data:

In [1]:
import json
with open('grades.json', 'rb') as f:
    data = json.load(f)

Extracting the relevant information out of the json for one course:

In [2]:
build_dict = lambda course: {
    'id': course['content']['achievementDto']['cpCourseLibDto']['id'], 
    'course_name_de': course['content']['achievementDto']['cpCourseLibDto']['courseTitle']['value'], 
    'course_name_en': course['content']['achievementDto']['cpCourseLibDto']['courseTitle']['translations']['translation'][1]['value'], 
    'course_number': course['content']['achievementDto']['cpCourseLibDto']['courseNumber']['courseNumber'], 
    'ects': course['content']['achievementDto']['cpCourseLibDto']['ectsCredits'], 
    'semester_code': course['content']['achievementDto']['semesterLibDto']['key'], 
    'semester_name_de': course['content']['achievementDto']['semesterLibDto']['semesterDesignation']
    ['value'], 
    'semester_name_en': course['content']['achievementDto']['semesterLibDto']['semesterDesignation']
    ['translations']['translation'][1]['value'], 
    'semester_start_date': course['content']['achievementDto']['semesterLibDto']['startOfAcademicSemester']['value'], 
    'semester_end_date': course['content']['achievementDto']['semesterLibDto']['endOfAcademicSemester']['value'], 
    'grade_date': course['content']['achievementDto']['achievementDate']['value'], 
    'grade_name_de': course['content']['achievementDto']['gradeDto']['name']['value'], 
    'grade_name_en': course['content']['achievementDto']['gradeDto']['name']['translations']['translation'][1]['value'], 
    'grade': course['content']['achievementDto']['gradeDto']['value'], 
}

Creating a list of dicts, each dict containing the info for one course.

In [3]:
dicts = [build_dict(course) for course in data['resource']]

For each course, parse the grades out of its html file, and add to its dict:

In [4]:
from bs4 import BeautifulSoup

possible_grades = ['1.0', '1.3', '1.4', '1.7', '2.0', '2.3', '2.4', '2.7', '3.0', '3.3', '3.4', '3.7', '4.0', '4.3', '4.7', '5.0']
standard_possible_grades = ['1.0', '1.3', '1.7', '2.0', '2.3', '2.7', '3.0', '3.3', '3.7', '4.0', '4.3', '4.7', '5.0']
all_possible_grades = possible_grades + ['did_not_show_up']

for d in reversed(dicts): # iterating in reverse order so we can remove elements while iterating.
    # University regulation: written exams from first semester are weighted half the points.
    d['grade_weight'] = d['ects']
    if ('Discrete Structures' in d['course_name_en'] 
        or 'Introduction to Informatics' in d['course_name_en']
        or 'Computer Organization' in d['course_name_en']
       ):
        d['grade_weight'] >>= 1 # divide by 2 but leave as int (known to all be devisable by 2)
    # read the html file to a string
    try:
        with open('stats/{}.html'.format(d['id']), 'rb') as f:
            html_doc = f.read()    
        soup = BeautifulSoup(html_doc, 'html.parser')
        # the data can be found in the titles of div objects with the class "kandcountbox"
        divs = soup.find_all('div', 'kandcountbox')
        titles = [div['title'] for div in divs]
        # A list of tuples (<grade>, <number of students>) e.g. ('1.0', 3)
        nums = [(ts[-1].split()[0], int(ts[-2].split()[0])) for t in titles if (ts := t.split(','))]
        d.update((grade, 0) for grade in all_possible_grades) # All courses get all grades, also if 0 students that grade.
        for i, t in enumerate(titles):
            if 'Nicht erschienen' in t: # Students who did not show up
                d['did_not_show_up'] = nums[i][1]
            elif '5.0' in t: # add up fails and cheats together.
                d['5.0'] += nums[i][1]
        # We already counted all the 5.0s and added them, so don't add again.
        d.update((tup for tup in nums if tup[0] != '5.0'))
    except FileNotFoundError:
        print("No statistics file for ", d['course_name_en'])
        dicts.remove(d)


Create a pandas dataframe with the data:

In [5]:
import pandas as pd

df = pd.DataFrame(dicts)
df['did_show_up'] = df[possible_grades].sum(axis=1)
df['numeric_grade'] = pd.to_numeric(df['grade'])
df['int_grade_X10'] = df['grade'].apply(lambda x: int((x.replace('.', '') + '0')[:2]))
df['5.0_with_noshows'] = df['5.0'] + df['did_not_show_up']
df['total_students'] = df['did_show_up'] + df['did_not_show_up']
df['grade'] = df['grade'].apply(lambda s: (s + '.0')[:3])
grades_with_noshows = possible_grades + ['5.0_with_noshows']
grades_with_noshows.remove('5.0')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

,id,course_name_de,course_name_en,course_number,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,grade_date,grade_name_de,grade_name_en,grade,grade_weight,1.0,1.3,1.4,1.7,2.0,2.3,2.4,2.7,3.0,3.3,3.4,3.7,4.0,4.3,4.7,5.0,did_not_show_up,did_show_up,numeric_grade,int_grade_X10,5.0_with_noshows,total_students
0,950560131,Systems Programming in C++,Systems Programming in C++,IN4256,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-09-14T00:00:00,sehr gut,very good,1.0,10,14,7,0,2,2,1,0,3,2,1,0,0,0,0,0,4,0,36,1.0,10,4,36
1,950552753,Compilerbau I,Compiler Construction I,IN2227,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-26T00:00:00,gut,good,2.0,5,9,10,0,15,18,15,0,22,31,13,0,12,7,8,0,1,40,161,2.0,20,41,201
2,950548250,Statistik für BWL (Einführung mit R),Statistics for Business Administration,MA9712M,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-22T00:00:00,befriedigend,satisfactory,2.7,6,11,23,0,28,59,103,0,125,114,97,0,70,35,39,14,34,68,752,2.7,27,102,820
3,950549876,Computergestützte Statistik,Computational Statistics,MA3402,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,2021-07-20T00:00:00,ausreichend,sufficient,4.0,5,3,3,0,9,14,10,0,9,14,12,0,10,13,18,19,29,72,163,4.0,40,101,235
4,950511694,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,IN2062,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-03-01T00:00:00,befriedigend,satisfactory,3.0,5,20,29,0,44,66,70,0,77,86,69,0,83,65,61,39,107,454,816,3.0,30,561,1270
5,950525218,Höhere Algorithmik,Advanced Algorithms,IN2360,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,2021-02-23T00:00:00,befriedigend,satisfactory,2.7,6,8,4,0,4,6,1,0,2,9,6,0,4,2,7,2,11,36,66,2.7,27,47,102
6,950478816,Blockkurs Spanisch A1,Intensive Course Spanish A1,SZ1201-1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-09-09T00:00:00,sehr gut,very good,1.0,3,3,7,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,16,1.0,10,0,16
7,950487012,Modellbildung und Simulation (Fokus Analysis),Modelling and Simulation (Focus Analysis),IN2366,9,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-08-14T00:00:00,befriedigend,satisfactory,2.7,9,4,3,0,4,3,5,0,7,8,1,0,6,1,1,0,5,14,48,2.7,27,19,62
8,950475386,Einführung in die Theoretische Informatik,Introduction to Theory of Computation,IN0011,8,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-08-13T00:00:00,sehr gut,very good,1.0,8,42,15,0,21,23,35,0,33,35,57,0,40,96,68,76,135,181,676,1.0,10,316,857
9,950474800,Diskrete Wahrscheinlichkeitstheorie,Discrete Probability Theory,IN0018,6,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,2020-08-05T00:00:00,befriedigend,satisfactory,2.7,6,39,15,0,15,24,30,0,41,45,47,0,51,15,61,48,126,148,557,2.7,27,274,705


In [6]:
int_grades = list(map(lambda x: int(x.replace('.', '')), possible_grades)) # '1.3' -> 13
df['mean'] = ((df[possible_grades] * int_grades).sum(axis=1) / df['did_show_up']) / 10
df['mean_with_noshows'] = ((df[grades_with_noshows] * int_grades).sum(axis=1) / df['total_students']) / 10
df

,id,course_name_de,course_name_en,course_number,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,...,4.7,5.0,did_not_show_up,did_show_up,numeric_grade,int_grade_X10,5.0_with_noshows,total_students,mean,mean_with_noshows
0,950560131,Systems Programming in C++,Systems Programming in C++,IN4256,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,0,4,0,36,1.0,10,4,36,1.950000,1.950000
1,950552753,Compilerbau I,Compiler Construction I,IN2227,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,0,1,40,161,2.0,20,41,201,2.640373,3.109950
2,950548250,Statistik für BWL (Einführung mit R),Statistics for Business Administration,MA9712M,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,14,34,68,752,2.7,27,102,820,2.986037,3.153049
3,950549876,Computergestützte Statistik,Computational Statistics,MA3402,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,19,29,72,163,4.0,40,101,235,3.557055,3.999149
4,950511694,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,IN2062,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,...,39,107,454,816,3.0,30,561,1270,3.268137,3.887244
5,950525218,Höhere Algorithmik,Advanced Algorithms,IN2360,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,...,2,11,36,66,2.7,27,47,102,3.087879,3.762745
6,950478816,Blockkurs Spanisch A1,Intensive Course Spanish A1,SZ1201-1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,0,0,0,16,1.0,10,0,16,1.393750,1.393750
7,950487012,Modellbildung und Simulation (Fokus Analysis),Modelling and Simulation (Focus Analysis),IN2366,9,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,0,5,14,48,2.7,27,19,62,2.789583,3.288710
8,950475386,Einführung in die Theoretische Informatik,Introduction to Theory of Computation,IN0011,8,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,76,135,181,676,1.0,10,316,857,3.642751,3.929405
9,950474800,Diskrete Wahrscheinlichkeitstheorie,Discrete Probability Theory,IN0018,6,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,48,126,148,557,2.7,27,274,705,3.533932,3.841702


In [7]:
import numpy as np

We want to calculate the Z-Score of each grade. For a single course, where I got the grade $x$, the mean grade is $\mu$ and the standard deviation is $\sigma$, then $Z = \frac{x - \mu}{\sigma}$. We now calculate that simulataneously for each course.

We already have the mean and the grade of each course, so now we calculate the standard deviation. For a single course we used the (biased) standard deviation $$\sigma = \sqrt{\frac{\sum_{x \in X}(x-\mu)^2}{N}}$$ where $X$ is the the set of all grades in this one exam and $N$ is the total number of students.
Our data looks a bit different however. What we have is for every possible grade, the number of students with that grade, so our calculation will look like this: 
$$\sigma = \sqrt{\frac{\sum_{g \in G}n_g(g-\mu)^2}{N}}$$
Where $G$ is the set of possible grades ($1.0$, $1.3$, ...) and for a grade $g \in G$, $n_g$ is the number of students with that grade in this exam.

First, we create a matrix which has for each exam the row vector $(\ \ldots \ (g - \mu)^2 \   \ldots\ )_{g \in G}$ which contain the squared difference of every possible grade and the mean:

In [8]:
# The (x - \mu)^2 part of the calculation for the sd.
squared_differences = np.square(np.array([np.array(int_grades)/10,]*len(df)) - np.array([df['mean'],] * len(int_grades)).transpose())

We now multiply this matrix element-wise with our data, and get the matrix 
$(n_{g,i}(g-\mu)^2)_{g \in G, i \in E}$ where $E$ is the set of exams and $n_{g,i}$ is the number of students who get the grade $g$ in exam $i$. We then sum each row, and divide it by the total number of students who took that exam, which gives us a vector where every element is the complete term inside the squre root above, for one exam. We then just take the square root element-wise, and get a vector of the standard deviations of all exams, which we add to our DataFrame.

In [9]:
df['standard_deviation'] = np.sqrt(np.multiply(df[possible_grades], squared_differences).sum(axis=1) / df['did_show_up'])

In [10]:
df

,id,course_name_de,course_name_en,course_number,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,...,5.0,did_not_show_up,did_show_up,numeric_grade,int_grade_X10,5.0_with_noshows,total_students,mean,mean_with_noshows,standard_deviation
0,950560131,Systems Programming in C++,Systems Programming in C++,IN4256,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,4,0,36,1.0,10,4,36,1.950000,1.950000,1.277693
1,950552753,Compilerbau I,Compiler Construction I,IN2227,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,1,40,161,2.0,20,41,201,2.640373,3.109950,0.892785
2,950548250,Statistik für BWL (Einführung mit R),Statistics for Business Administration,MA9712M,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,34,68,752,2.7,27,102,820,2.986037,3.153049,0.892316
3,950549876,Computergestützte Statistik,Computational Statistics,MA3402,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,29,72,163,4.0,40,101,235,3.557055,3.999149,1.175724
4,950511694,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,IN2062,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,...,107,454,816,3.0,30,561,1270,3.268137,3.887244,1.130250
5,950525218,Höhere Algorithmik,Advanced Algorithms,IN2360,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,...,11,36,66,2.7,27,47,102,3.087879,3.762745,1.362875
6,950478816,Blockkurs Spanisch A1,Intensive Course Spanish A1,SZ1201-1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,0,0,16,1.0,10,0,16,1.393750,1.393750,0.260933
7,950487012,Modellbildung und Simulation (Fokus Analysis),Modelling and Simulation (Focus Analysis),IN2366,9,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,5,14,48,2.7,27,19,62,2.789583,3.288710,1.128280
8,950475386,Einführung in die Theoretische Informatik,Introduction to Theory of Computation,IN0011,8,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,135,181,676,1.0,10,316,857,3.642751,3.929405,1.219888
9,950474800,Diskrete Wahrscheinlichkeitstheorie,Discrete Probability Theory,IN0018,6,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,126,148,557,2.7,27,274,705,3.533932,3.841702,1.269960


Once we have all that, we just do the $\frac{x-\mu}{\sigma}$ calculation for each exam.

Of course we do it with numpy's vectorized operations.

In [11]:
df['z'] = (df['numeric_grade'] - df['mean']) / df['standard_deviation']
df

,id,course_name_de,course_name_en,course_number,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,...,did_not_show_up,did_show_up,numeric_grade,int_grade_X10,5.0_with_noshows,total_students,mean,mean_with_noshows,standard_deviation,z
0,950560131,Systems Programming in C++,Systems Programming in C++,IN4256,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,0,36,1.0,10,4,36,1.950000,1.950000,1.277693,-0.743527
1,950552753,Compilerbau I,Compiler Construction I,IN2227,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,40,161,2.0,20,41,201,2.640373,3.109950,0.892785,-0.717275
2,950548250,Statistik für BWL (Einführung mit R),Statistics for Business Administration,MA9712M,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,68,752,2.7,27,102,820,2.986037,3.153049,0.892316,-0.320556
3,950549876,Computergestützte Statistik,Computational Statistics,MA3402,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,72,163,4.0,40,101,235,3.557055,3.999149,1.175724,0.376742
4,950511694,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,IN2062,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,...,454,816,3.0,30,561,1270,3.268137,3.887244,1.130250,-0.237237
5,950525218,Höhere Algorithmik,Advanced Algorithms,IN2360,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,...,36,66,2.7,27,47,102,3.087879,3.762745,1.362875,-0.284603
6,950478816,Blockkurs Spanisch A1,Intensive Course Spanish A1,SZ1201-1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,0,16,1.0,10,0,16,1.393750,1.393750,0.260933,-1.509009
7,950487012,Modellbildung und Simulation (Fokus Analysis),Modelling and Simulation (Focus Analysis),IN2366,9,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,14,48,2.7,27,19,62,2.789583,3.288710,1.128280,-0.079398
8,950475386,Einführung in die Theoretische Informatik,Introduction to Theory of Computation,IN0011,8,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,181,676,1.0,10,316,857,3.642751,3.929405,1.219888,-2.166388
9,950474800,Diskrete Wahrscheinlichkeitstheorie,Discrete Probability Theory,IN0018,6,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,148,557,2.7,27,274,705,3.533932,3.841702,1.269960,-0.656660


In [12]:
# Now compute Z-score with noshows:
squared_differences = np.square(np.array([np.array(int_grades)/10,]*len(df)) - np.array([df['mean_with_noshows'],] * len(int_grades)).transpose())
df['standard_deviation_with_noshows'] = np.sqrt(np.multiply(df[grades_with_noshows], squared_differences).sum(axis=1) / df['total_students'])
df['z_with_noshows'] = (df['numeric_grade'] - df['mean_with_noshows']) / df['standard_deviation_with_noshows']
df

,id,course_name_de,course_name_en,course_number,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,...,numeric_grade,int_grade_X10,5.0_with_noshows,total_students,mean,mean_with_noshows,standard_deviation,z,standard_deviation_with_noshows,z_with_noshows
0,950560131,Systems Programming in C++,Systems Programming in C++,IN4256,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,1.0,10,4,36,1.950000,1.950000,1.277693,-0.743527,1.277693,-0.743527
1,950552753,Compilerbau I,Compiler Construction I,IN2227,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,2.0,20,41,201,2.640373,3.109950,0.892785,-0.717275,1.235302,-0.898526
2,950548250,Statistik für BWL (Einführung mit R),Statistics for Business Administration,MA9712M,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,2.7,27,102,820,2.986037,3.153049,0.892316,-0.320556,1.019147,-0.444537
3,950549876,Computergestützte Statistik,Computational Statistics,MA3402,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,4.0,40,101,235,3.557055,3.999149,1.175724,0.376742,1.183755,0.000719
4,950511694,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,IN2062,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,...,3.0,30,561,1270,3.268137,3.887244,1.130250,-0.237237,1.228703,-0.722098
5,950525218,Höhere Algorithmik,Advanced Algorithms,IN2360,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,...,2.7,27,47,102,3.087879,3.762745,1.362875,-0.284603,1.427182,-0.744646
6,950478816,Blockkurs Spanisch A1,Intensive Course Spanish A1,SZ1201-1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,1.0,10,0,16,1.393750,1.393750,0.260933,-1.509009,0.260933,-1.509009
7,950487012,Modellbildung und Simulation (Fokus Analysis),Modelling and Simulation (Focus Analysis),IN2366,9,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,2.7,27,19,62,2.789583,3.288710,1.128280,-0.079398,1.356360,-0.434037
8,950475386,Einführung in die Theoretische Informatik,Introduction to Theory of Computation,IN0011,8,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,1.0,10,316,857,3.642751,3.929405,1.219888,-2.166388,1.216849,-2.407369
9,950474800,Diskrete Wahrscheinlichkeitstheorie,Discrete Probability Theory,IN0018,6,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,2.7,27,274,705,3.533932,3.841702,1.269960,-0.656660,1.276994,-0.894054


In [13]:
# inclusive definition of percentile
df['percentile'] = (np.multiply(np.array([np.array(int_grades),]*len(df)) < np.array([df['int_grade_X10'],] * len(int_grades)).transpose(), df[possible_grades]).sum(axis=1) / df['did_show_up']) * 100

In [14]:
# calculate percentile including noshows
df['percentile_with_noshows'] = (np.multiply(np.array([np.array(int_grades),]*len(df)) < np.array([df['int_grade_X10'],] * len(int_grades)).transpose(), df[grades_with_noshows]).sum(axis=1) / df['total_students']) * 100
df

,id,course_name_de,course_name_en,course_number,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,...,5.0_with_noshows,total_students,mean,mean_with_noshows,standard_deviation,z,standard_deviation_with_noshows,z_with_noshows,percentile,percentile_with_noshows
0,950560131,Systems Programming in C++,Systems Programming in C++,IN4256,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,4,36,1.950000,1.950000,1.277693,-0.743527,1.277693,-0.743527,0.000000,0.000000
1,950552753,Compilerbau I,Compiler Construction I,IN2227,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,41,201,2.640373,3.109950,0.892785,-0.717275,1.235302,-0.898526,21.118012,16.915423
2,950548250,Statistik für BWL (Einführung mit R),Statistics for Business Administration,MA9712M,6,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,102,820,2.986037,3.153049,0.892316,-0.320556,1.019147,-0.444537,29.787234,27.317073
3,950549876,Computergestützte Statistik,Computational Statistics,MA3402,5,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,101,235,3.557055,3.999149,1.175724,0.376742,1.183755,0.000719,51.533742,35.744681
4,950511694,Grundlagen der Künstlichen Intelligenz,Techniques in Artificial Intelligence,IN2062,5,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,...,561,1270,3.268137,3.887244,1.130250,-0.237237,1.228703,-0.722098,37.500000,24.094488
5,950525218,Höhere Algorithmik,Advanced Algorithms,IN2360,6,20W,Wintersemester 2020/21,Winter semester 2020/21,2020-10-01,2021-03-31,...,47,102,3.087879,3.762745,1.362875,-0.284603,1.427182,-0.744646,34.848485,22.549020
6,950478816,Blockkurs Spanisch A1,Intensive Course Spanish A1,SZ1201-1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,0,16,1.393750,1.393750,0.260933,-1.509009,0.260933,-1.509009,0.000000,0.000000
7,950487012,Modellbildung und Simulation (Fokus Analysis),Modelling and Simulation (Focus Analysis),IN2366,9,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,19,62,2.789583,3.288710,1.128280,-0.079398,1.356360,-0.434037,39.583333,30.645161
8,950475386,Einführung in die Theoretische Informatik,Introduction to Theory of Computation,IN0011,8,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,316,857,3.642751,3.929405,1.219888,-2.166388,1.216849,-2.407369,0.000000,0.000000
9,950474800,Diskrete Wahrscheinlichkeitstheorie,Discrete Probability Theory,IN0018,6,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,274,705,3.533932,3.841702,1.269960,-0.656660,1.276994,-0.894054,22.082585,17.446809


In [15]:
# mean Z score
mean_z_score = np.multiply(df['z'], df['grade_weight']).sum() / df['grade_weight'].sum()
print(mean_z_score)
mean_z_score_with_noshows = np.multiply(df['z_with_noshows'], df['grade_weight']).sum() / df['grade_weight'].sum()
print(mean_z_score_with_noshows)

-0.8910205278793454
-1.0571721224499904


In [16]:
avg_grade = np.multiply(df['numeric_grade'], df['grade_weight']).sum() / df['grade_weight'].sum()
print(avg_grade)

1.9315789473684213


In [17]:
# Percentile is weighted by the grade weight and by the number of students in the course.
weights = np.multiply(df['did_show_up'], df['grade_weight'])
avg_percentile = np.multiply(df['percentile'], weights).sum() / weights.sum()
print(avg_percentile)
weights = np.multiply(df['total_students'], df['grade_weight'])
avg_percentile_with_noshows = np.multiply(df['percentile_with_noshows'], weights).sum() / weights.sum()
print(avg_percentile_with_noshows)

13.616493315244488
11.458373990086258


In [18]:
total_ects = df['ects'].sum()

In [19]:
df.sort_values(by=['grade','ects', 'percentile', 'z', 'semester_start_date', 'course_name_en'], 
               ascending=[True, False, True, True, True, True], inplace=True, )
df

,id,course_name_de,course_name_en,course_number,ects,semester_code,semester_name_de,semester_name_en,semester_start_date,semester_end_date,...,5.0_with_noshows,total_students,mean,mean_with_noshows,standard_deviation,z,standard_deviation_with_noshows,z_with_noshows,percentile,percentile_with_noshows
0,950560131,Systems Programming in C++,Systems Programming in C++,IN4256,10,21S,Sommersemester 2021,Summer semester 2021,2021-04-01,2021-09-30,...,4,36,1.950000,1.950000,1.277693,-0.743527,1.277693,-0.743527,0.000000,0.000000
8,950475386,Einführung in die Theoretische Informatik,Introduction to Theory of Computation,IN0011,8,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,316,857,3.642751,3.929405,1.219888,-2.166388,1.216849,-2.407369,0.000000,0.000000
23,950383197,Praktikum Grundlagen der Programmierung,Fundamentals of Programming (Exercises & Labor...,IN0002,6,18W,Wintersemester 2018/19,Winter semester 2018/19,2018-10-01,2019-03-31,...,255,1447,3.199444,3.208155,1.403789,-1.566791,1.405951,-1.570577,0.000000,0.000000
15,950442006,Grundlagen: Datenbanken,Fundamentals of Databases,IN0008,6,19W,Wintersemester 2019/20,Winter semester 2019/20,2019-10-01,2020-03-31,...,341,1214,2.860829,3.299588,1.199585,-1.551227,1.374746,-1.672737,0.000000,0.000000
16,950442110,Funktionale Programmierung und Verifikation,Functional Programming and Verification,IN0003,5,19W,Wintersemester 2019/20,Winter semester 2019/20,2019-10-01,2020-03-31,...,243,868,3.291712,3.551498,1.209361,-1.894978,1.271392,-2.006854,0.000000,0.000000
6,950478816,Blockkurs Spanisch A1,Intensive Course Spanish A1,SZ1201-1,3,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,0,16,1.393750,1.393750,0.260933,-1.509009,0.260933,-1.509009,0.000000,0.000000
26,950394428,Deutsch im Bachelorstudium C1 - Informatik: Wi...,German for Bachelor's Students C1 - Informatic...,SZ0324,3,18W,Wintersemester 2018/19,Winter semester 2018/19,2018-10-01,2019-03-31,...,2,25,1.126087,1.436000,0.301038,-0.418841,1.089910,-0.400033,0.000000,0.000000
13,950442112,Grundlagen: Betriebssysteme und Systemsoftware,Basic Principles: Operating Systems and System...,IN0009,6,19W,Wintersemester 2019/20,Winter semester 2019/20,2019-10-01,2020-03-31,...,198,852,3.021674,3.307277,1.202123,-1.432194,1.311459,-1.530568,6.584362,5.633803
11,950528080,Grundlagen: Rechnernetze und verteilte Systeme,Introduction to Computer Networking and Distri...,IN0010E,6,20S,Sommersemester 2020,Summer semester 2020,2020-04-01,2020-09-30,...,284,786,3.215097,3.601145,1.354076,-1.414320,1.406052,-1.636600,11.201299,8.778626
24,950383477,Einführung in die Informatik 1,Introduction to Informatics 1,IN0001,6,18W,Wintersemester 2018/19,Winter semester 2018/19,2018-10-01,2019-03-31,...,282,1446,3.452271,3.656708,1.097564,-1.596509,1.148976,-1.703002,5.258964,4.564315


Save the DataFrame data to a json:

In [20]:
df.to_json(path_or_buf='web/js/grade_data.json', orient='records')

In [21]:
df[possible_grades + ['grade_weight']]

,1.0,1.3,1.4,1.7,2.0,2.3,2.4,2.7,3.0,3.3,3.4,3.7,4.0,4.3,4.7,5.0,grade_weight
0,14,7,0,2,2,1,0,3,2,1,0,0,0,0,0,4,10
8,42,15,0,21,23,35,0,33,35,57,0,40,96,68,76,135,8
23,197,67,0,60,77,81,0,93,98,108,0,97,78,102,134,248,6
15,98,3,49,71,91,18,74,108,88,44,38,73,42,42,34,92,6
16,39,23,0,40,57,50,0,55,67,74,0,69,59,39,53,111,5
6,3,7,0,6,0,0,0,0,0,0,0,0,0,0,0,0,3
26,18,3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,3
13,48,39,0,49,60,60,0,92,65,79,0,36,23,64,39,75,6
11,69,23,0,31,32,47,0,41,51,58,0,44,15,47,44,114,6
24,45,21,0,60,83,78,0,73,99,110,0,108,131,240,116,91,3


In [22]:
# multiply the number of students that got each grade, with the weight of that exam.
weighted_nums = (df[possible_grades].transpose() * df['grade_weight']).transpose()
weighted_nums

,1.0,1.3,1.4,1.7,2.0,2.3,2.4,2.7,3.0,3.3,3.4,3.7,4.0,4.3,4.7,5.0
0,140,70,0,20,20,10,0,30,20,10,0,0,0,0,0,40
8,336,120,0,168,184,280,0,264,280,456,0,320,768,544,608,1080
23,1182,402,0,360,462,486,0,558,588,648,0,582,468,612,804,1488
15,588,18,294,426,546,108,444,648,528,264,228,438,252,252,204,552
16,195,115,0,200,285,250,0,275,335,370,0,345,295,195,265,555
6,9,21,0,18,0,0,0,0,0,0,0,0,0,0,0,0
26,54,9,0,3,0,3,0,0,0,0,0,0,0,0,0,0
13,288,234,0,294,360,360,0,552,390,474,0,216,138,384,234,450
11,414,138,0,186,192,282,0,246,306,348,0,264,90,282,264,684
24,135,63,0,180,249,234,0,219,297,330,0,324,393,720,348,273


In [23]:
sums = weighted_nums.sum()
sums

1.0     6081
1.3     3073
1.4      492
1.7     4613
2.0     6109
2.3     6084
2.4      648
2.7     7193
3.0     7156
3.3     6747
3.4      756
3.7     6829
4.0     5051
4.3     8856
4.7     6211
5.0    10342
dtype: int64

In [24]:
denominator = sums.sum()
denominator

86241

In [25]:
mean = (sums * int_grades).sum() / (denominator * 10)
mean

3.2067299776208533

In [26]:
grades_array = np.array(int_grades)/10
squared_diffs = np.square(grades_array - mean)
squared_diffs

array([4.86965719, 3.63561921, 3.26427321, 2.27023523, 1.45619724,
       0.82215925, 0.65081326, 0.25677527, 0.04273728, 0.0086993 ,
       0.0373533 , 0.24331531, 0.62927733, 1.19523934, 2.22985536,
       3.21581737])

In [27]:
# element wise multiplication, to weight each diff with its grade weight, and count it for every student with that grade
weighted_squared_diffs = np.multiply(sums, squared_diffs)
weighted_squared_diffs

1.0    29612.385398
1.3    11172.257825
1.4     1606.022420
1.7    10472.595095
2.0     8895.908932
2.3     5002.016891
2.4      421.726990
2.7     1846.984519
3.0      305.828002
3.3       58.694157
3.4       28.239096
3.7     1661.600286
4.0     3178.479786
4.3    10585.039611
4.7    13849.631639
5.0    33257.983273
dtype: float64

In [28]:
from math import sqrt
sd = sqrt(weighted_squared_diffs.sum() / denominator)
sd

1.2369629222070873

In [29]:
total_z = (avg_grade - mean) / sd
total_z

-1.0308724759325902

This "total Z-Score" is calculated to take into account the number of students in each course, unlike when just taking an average (weighted for credits but not for students) of the Z-Scores of all courses. This result is less affected by courses with a small amount of participants. This might make sense if you are trying to compare me to the total population of computer science students. A course with a small number of participants could be like a study with a small number of participants. It is less likely to provide a selection representative of the entire population. In the "Mean Z-Score" we only weight courses by their ECTS, here we also take the number of participants into account.

In [30]:
# Calculate total Z again with noshows:
weighted_nums_noshows = (df[grades_with_noshows].transpose() * df['grade_weight']).transpose()
sums_noshows = weighted_nums_noshows.sum()
denominator_noshows = sums_noshows.sum()
mean_noshows = (sums_noshows * int_grades).sum() / (denominator_noshows * 10)
grades_array = np.array(int_grades)/10
squared_diffs_noshows = np.square(grades_array - mean_noshows)
weighted_squared_diffs_noshows = np.multiply(sums_noshows, squared_diffs_noshows)
sd_noshows = sqrt(weighted_squared_diffs_noshows.sum() / denominator_noshows)
total_z_with_noshows = (avg_grade - mean_noshows) / sd_noshows
total_z_with_noshows

-1.1902305357688263

In [31]:
with open('web/js/agg_data.json', 'w') as agg_f:
    json.dump({
        'mean_z_score': total_z,
        'mean_z_score_with_noshows': total_z_with_noshows,
        'avg_grade': avg_grade,
        'avg_percentile': avg_percentile,
        'avg_percentile_with_noshows': avg_percentile_with_noshows,
        'total_ects': int(total_ects),
    }, agg_f)

In [32]:
def aggregate_semester(group_frame):
    d = {}
    d['Number of Courses'] = len(group_frame.index)
    d['Total ECTS'] = group_frame['ects'].sum()
    d['My Mean Grade'] = np.average(group_frame.numeric_grade, weights=group_frame.grade_weight)
    d['My Mean Z-Score'] = np.average(group_frame.z, weights=np.multiply(group_frame.grade_weight, group_frame.did_show_up))
    d['My Mean Percentile'] = np.average(group_frame.percentile, weights=np.multiply(group_frame.grade_weight, group_frame.did_show_up))
    # Using dtype object to prevent int to float conversion. 
    # It's not that bad because no calculation will be mad on the series in that state,
    # And once back in the dataframe, each column gets a specific type
    return pd.Series(d, dtype=object)


semester_data = df.groupby(['semester_start_date', 'semester_code'] # include code for display, date for sort
                          ).apply(aggregate_semester
                                 ).sort_values('semester_start_date' # probably already sorted, but to be safe.
                                              ).reset_index().drop('semester_start_date', axis=1)
semester_data

,semester_code,Number of Courses,Total ECTS,My Mean Grade,My Mean Z-Score,My Mean Percentile
0,18W,5,31,2.045000,-1.151321,10.451378
1,19S,4,28,1.850000,-0.997573,15.431123
2,19W,6,34,1.561765,-1.428789,6.782807
3,20S,6,37,1.832432,-1.486196,10.250077
4,20W,2,11,2.836364,-0.241428,37.265416
5,21S,4,26,2.161538,-0.305665,29.790511


In [33]:
# Check that the types make sense:
semester_data.dtypes

semester_code          object
Number of Courses       int64
Total ECTS              int64
My Mean Grade         float64
My Mean Z-Score       float64
My Mean Percentile    float64
dtype: object

In [34]:
with open('web/js/semester_data.json', 'w') as semester_f:
    json.dump(semester_data.to_dict('list'), semester_f)

In [35]:
my_grade_distribution = df[['grade', 'grade_weight']].groupby('grade').sum()
my_grade_distribution['grade_weight'] = my_grade_distribution['grade_weight'] / my_grade_distribution['grade_weight'].sum()
empty_frame = pd.DataFrame(0, index=standard_possible_grades, columns=['grade_weight'], dtype=my_grade_distribution.dtypes[0])
complete_distribution = pd.merge(my_grade_distribution, empty_frame, left_index=True, right_index=True, how='outer', sort=True).fillna(0)['grade_weight_x']
complete_distribution

1.0    0.269737
1.3    0.078947
1.7    0.171053
2.0    0.164474
2.3    0.000000
2.7    0.197368
3.0    0.059211
3.3    0.000000
3.7    0.026316
4.0    0.032895
4.3    0.000000
4.7    0.000000
5.0    0.000000
Name: grade_weight_x, dtype: float64

In [36]:
with open('web/js/personal_distribution.json', 'w') as dist_f:
    json.dump(list(complete_distribution), dist_f)